In [1]:
#Dual Encryption using AES and Blowfish. The key of AES is encrypted using RSA.

In [2]:
import time
start_time = time.time()
import numpy as np
import blowfish
import rsa
from Cryptodome.Cipher import AES
from PIL import Image
import cv2
import math

%matplotlib auto

Using matplotlib backend: Qt5Agg


In [3]:
image = cv2.imread('bird.jpg',1)
cv2.imshow('original', image)

In [4]:
img_str = cv2.imencode('.jpg', image)[1].tobytes()

In [5]:
if len(img_str)%8 != 0:#check for compatibility
    while len(img_str)%8 != 0:
        img_str += b'\x00'

og_img_np = np.frombuffer(img_str, dtype=np.uint8)

In [6]:
key = b'aespasswordisnow'
cipher_a = AES.new(key, AES.MODE_EAX)
nonce = cipher_a.nonce
ciphertext_enc, tag = cipher_a.encrypt_and_digest(img_str)

In [26]:
public_key, private_key = rsa.newkeys(1024)
public_key

PublicKey(91179494065749113790696725352224318842046410494010704463030324452223689808090373157219341259011143310424726376197654449431086689391379657498966610979976228150448985095592705482043875484574132152602483320494269056977345382539191291937555770529543434113056114545444594456154090636250267342688723692172397521527, 65537)

In [8]:
sig = rsa.sign(key, private_key, 'SHA-256')
enc_key = rsa.encrypt(key, public_key)

In [9]:
img_str_np_enc_aes = np.frombuffer(ciphertext_enc, dtype=np.uint8)
a = len(img_str_np_enc_aes)
i = 1
while a%2 == 0:
    i = i*2
    a = a//2


img_str_np_enc_aes1 = img_str_np_enc_aes.reshape(i,len(img_str_np_enc_aes)//i)

In [10]:
enc_img_aes = Image.fromarray(img_str_np_enc_aes1, 'L')
enc_img_aes.show(title='aes encrypted')

In [11]:
cipher_b = blowfish.Cipher(b'blowfishpasswordishere')

In [12]:
data = img_str # electronic codebook mode

data_encrypted = b"".join(cipher_b.encrypt_ecb(ciphertext_enc))

In [13]:
img_str_np_enc_blow = np.frombuffer(data_encrypted, dtype=np.uint8)
a = len(img_str_np_enc_blow)
i = 1
while a%2 == 0:
    i = i*2
    a = a//2


img_str_np_enc_blow1 = img_str_np_enc_blow.reshape(i,len(img_str_np_enc_blow)//i)

In [14]:
enc_img_blow = Image.fromarray(img_str_np_enc_blow1, 'L')
enc_img_blow.show(title='blowfish encrypted')

In [15]:
data_decrypted = b"".join(cipher_b.decrypt_ecb(data_encrypted))

#assert data == data_decrypted

In [16]:
dec_key = rsa.decrypt(enc_key, private_key)
rsa.verify(dec_key, sig, public_key)

'SHA-256'

In [17]:
cipher_a = AES.new(dec_key, AES.MODE_EAX, nonce=nonce)
plaintext_enc = cipher_a.decrypt(data_decrypted)
try:
    cipher_a.decrypt(ciphertext_enc)
    print("AES decryption successful")
except ValueError:
    print("Decryption failed")

AES decryption successful


In [18]:
img_str_np = np.frombuffer(plaintext_enc, dtype=np.uint8)
img_str_np

array([255, 216, 255, ...,   0,   0,   0], dtype=uint8)

In [19]:
img = cv2.imdecode(img_str_np, 1)

In [20]:
cv2.imshow('decrypted', img)

In [21]:
def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

d = psnr(image, img)
print(d)

43.22586472210333


In [22]:
cv2.PSNR(image, img)

42.722213649793986

In [23]:
Y = np.square(np.subtract(image,img)).mean()
print("MSE:", Y)

MSE: 3.0938117793787896


In [24]:
np.corrcoef(image.ravel(), img.ravel())

array([[1.        , 0.99943421],
       [0.99943421, 1.        ]])

In [25]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 11.646512269973755 seconds ---
